In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
data_path = '/content/gdrive/My Drive/ENERGY_CLIMATE_HACK/raw_data/'

In [4]:
df = pd.read_csv(f'{data_path}booster_challenge_E.csv')

In [5]:
print(f'number of rows in df is {df.shape[0]}, number of cols in df is {df.shape[1]}')

number of rows in df is 2021, number of cols in df is 31


In [6]:
df.columns

Index(['region_id', 'region_name', 'percentage_population_in_detached_houses',
       'population', 'population_per_km2', 'percentage_detached_houses',
       'percentage_flats_3to4_rooms', 'municipality_type',
       'permanent_resident_population', 'swiss_citizen', 'foreign_citizen',
       'age_0to14', 'age_15to19', 'age_20to39', 'age_40to59', 'age_60to64',
       'age_65tomore', 'men', 'women', 'district_number', 'district_name',
       'kanton_number', 'kanton_short', 'kanton_name',
       'taxable_income_in_mio_CHF', 'taxable_income_per_taxable_person_in_chf',
       'share_electric_cars', 'share_pv', 'share_renewable_heating',
       'Heating data coverage', 'mean_electricity_tarif_2021_rp_per_kwh'],
      dtype='object')

In [7]:
for i in df.columns:
  print(f'feature name: {i} \t feature data type: {df[i].dtype}')

feature name: region_id 	 feature data type: int64
feature name: region_name 	 feature data type: object
feature name: percentage_population_in_detached_houses 	 feature data type: float64
feature name: population 	 feature data type: int64
feature name: population_per_km2 	 feature data type: float64
feature name: percentage_detached_houses 	 feature data type: float64
feature name: percentage_flats_3to4_rooms 	 feature data type: float64
feature name: municipality_type 	 feature data type: object
feature name: permanent_resident_population 	 feature data type: int64
feature name: swiss_citizen 	 feature data type: int64
feature name: foreign_citizen 	 feature data type: int64
feature name: age_0to14 	 feature data type: int64
feature name: age_15to19 	 feature data type: int64
feature name: age_20to39 	 feature data type: int64
feature name: age_40to59 	 feature data type: int64
feature name: age_60to64 	 feature data type: int64
feature name: age_65tomore 	 feature data type: int64


In [8]:
int_list = []
cat_list = []

for i in df.columns:
  if df[i].dtype != object:
    int_list.append(i)
  else:
    cat_list.append(i)

print('Numerical columns:')
print(int_list)
print('\n')
print('categorical columns:')
print(cat_list)

Numerical columns:
['region_id', 'percentage_population_in_detached_houses', 'population', 'population_per_km2', 'percentage_detached_houses', 'percentage_flats_3to4_rooms', 'permanent_resident_population', 'swiss_citizen', 'foreign_citizen', 'age_0to14', 'age_15to19', 'age_20to39', 'age_40to59', 'age_60to64', 'age_65tomore', 'men', 'women', 'district_number', 'kanton_number', 'taxable_income_in_mio_CHF', 'taxable_income_per_taxable_person_in_chf', 'share_electric_cars', 'share_pv', 'share_renewable_heating', 'Heating data coverage', 'mean_electricity_tarif_2021_rp_per_kwh']


categorical columns:
['region_name', 'municipality_type', 'district_name', 'kanton_short', 'kanton_name']


In [9]:
for i in df[cat_list]:
  print(df[i].value_counts())

Oberried am Brienzersee    1
Ruswil                     1
Adlikon                    1
Chavannes-le-Veyron        1
Mattstetten                1
                          ..
Veltheim (AG)              1
Schluein                   1
Crassier                   1
Froideville                1
Wäldi                      1
Name: region_name, Length: 2021, dtype: int64
Periurbane Gemeinde geringer Dichte (23)                                      446
Periurbane Gemeinde mittlerer Dichte (22)                                     368
Ländliche zentral gelegene Gemeinde (32)                                      353
Ländliche periphere Gemeinde (33)                                             194
Städtische Gemeinde einer mittelgrossen Agglomeration (12)                    193
Städtische Gemeinde einer grossen Agglomeration (11)                          160
Städtische Gemeinde einer kleinen oder ausserhalb einer Agglomeration (13)    120
Periurbane Gemeinde hoher Dichte (21)                        

In [10]:
# adding voting data

In [11]:
vote_df = pd.read_csv(f'{data_path}voting_outcome.csv', sep=';')

In [12]:
vote_df.columns

Index(['canton', 'yes', 'no', '% Ja', '% Nein', '% participation'], dtype='object')

In [13]:
#removing commas and percentage signs and coverting to numeric
numeric_cols = ['yes', 'no', '% Ja', '% Nein', '% participation']

vote_df = vote_df.apply(lambda x: x.str.replace("'", ''))
vote_df = vote_df.apply(lambda x: x.str.replace('%', ''))

for i in numeric_cols:
  vote_df[i] = vote_df[i].astype(float)

In [14]:
def accepted(x):
  if x['yes'] > x['no']:
    return 1
  return 0

vote_df['vote_accepted'] = vote_df.apply(lambda x: accepted(x), axis=1)

In [15]:
bin_map = {
    3: 'strongly for',
    2: 'marginally for',
    1: 'marginally against',
    0: 'strongly against',
}

vote_df['vote_bins'] = pd.cut(vote_df['% Ja'], bins=4, labels=False).map(bin_map)

In [16]:
vote_df[:3]

,canton,yes,no,% Ja,% Nein,% participation,vote_accepted,vote_bins
0,Zürich,311950.0,251032.0,55.41,44.59,59.46,1,marginally for
1,Bern,221956.0,235809.0,48.49,51.51,62.28,0,marginally against
2,Luzern,86846.0,94175.0,47.98,52.02,65.23,0,marginally against


In [17]:
# mergin to main dataset
# need to check canton names match

def clean_text(txt):
    txt = txt.replace('Bern', 'Bern / Berne')
    txt = txt.replace('Freiburg', 'Fribourg / Freiburg')
    txt = txt.replace('Genf', 'Genève')
    txt = txt.replace('Graubünden', 'Graubünden / Grigioni / Grischun')
    txt = txt.replace('Neuenburg', 'Neuchâtel')
    txt = txt.replace('Tessin', 'Ticino')
    txt = txt.replace('Wallis', 'Valais / Wallis')
    txt = txt.replace('Waadt', 'Vaud')
    return txt

In [18]:
vote_df['canton'] = vote_df['canton'].apply(clean_text)

In [19]:
set(df.kanton_name) - set(vote_df.canton)

set()

In [20]:
vote_df = vote_df.rename(columns={'canton':'kanton_name'})

In [21]:
df = df.merge(vote_df[['kanton_name', '% Ja', '% participation', 'vote_accepted', 'vote_bins']], how='left', on='kanton_name')

In [22]:
# adding spatial data for pv potential and temperature by district

In [23]:
spatial_df = pd.read_csv(f'{data_path}solar_potential_adm_2.csv')

In [24]:
spatial_df = spatial_df[(spatial_df['NAME_2'] !='Schaffhausen') | (spatial_df['NAME_1'] !='Zürich')]

In [25]:
spatial_df[spatial_df['NAME_2']=='Bernina']

,ID_0,ISO,NAME_0,ID_1,NAME_1,ID_2,NAME_2,TYPE_2,ENGTYPE_2,NL_NAME_2,VARNAME_2,temp_mean,temp_stdev,pv_out_mea,pv_out_std
57,223,CHE,Switzerland,10,Graubünden,58,Bernina,District,District,NaN,NaN,1.470175,3.461588,3.594847,0.34059


In [26]:
spatial_df[:3]

,ID_0,ISO,NAME_0,ID_1,NAME_1,ID_2,NAME_2,TYPE_2,ENGTYPE_2,NL_NAME_2,VARNAME_2,temp_mean,temp_stdev,pv_out_mea,pv_out_std
0,223,CHE,Switzerland,1,Aargau,1,Aarau,District,District,NaN,NaN,10.070621,0.442816,3.190780,0.025275
1,223,CHE,Switzerland,1,Aargau,2,Baden,District,District,NaN,NaN,10.066797,0.382364,3.211012,0.030895
2,223,CHE,Switzerland,1,Aargau,3,Bremgarten,District,District,NaN,NaN,10.172906,0.370601,3.239640,0.021938


In [27]:
def clean_text_main(txt):
  txt = txt.replace('Hinterland', 'Appenzell Ausserrhoden')
  txt = txt.replace('Mittelland', 'Appenzell Ausserrhoden')
  txt = txt.replace('Vorderland', 'Appenzell Ausserrhoden')
  txt = txt.replace('Bern-Appenzell Ausserrhoden', 'Appenzell Ausserrhoden')
  txt = txt.replace('Luzern-Stadt', 'Luzern-Land')
  return txt

df['district_name'] = df['district_name'].apply(clean_text_main)


In [28]:
import re

def clean_text_spatial(txt):
  #txt = txt.str.replace('Bern', 'Bern-Mittelland')
  txt = txt.replace('Biel', 'Biel/Bienne')
  txt = txt.replace('Genève', 'Canton de Genève')
  txt = txt.replace('Frutigen \r\n', 'Frutigen-Niedersimmental')
  txt = txt.replace('Burgdorf', 'Emmental')
  txt = txt.replace('Signau', 'Emmental')
  txt = txt.replace('Trachselwald', 'Emmental')
  txt = txt.replace('Interlaken', 'Interlaken-Oberhasli')
  txt = txt.replace('Courtelary', 'Jura bernois')
  txt = txt.replace('Moutier', 'Jura bernois')
  txt = txt.replace('La Neuveville', 'Jura bernois')
  txt = txt.replace('Jura-North Vaudois', 'Jura-Nord vaudois')
  txt = txt.replace('Appenzell Innerrhoden', 'Kanton Appenzell Innerrhoden')
  txt = txt.replace('Bettingen', 'Kanton Basel-Stadt')
  txt = txt.replace('Basel', 'Kanton Basel-Stadt')
  txt = txt.replace('Riehen', 'Kanton Basel-Stadt')
  txt = txt.replace('Glarus', 'Kanton Glarus')
  txt = txt.replace('Nidwalden', 'Kanton Nidwalden')
  txt = txt.replace('Uri', 'Kanton Uri')
  txt = txt.replace('Zug', 'Kanton Zug')
  txt = txt.replace('Küssnacht', 'Küssnacht (SZ)')
  txt = txt.replace('Luzern', 'Luzern-Land')
  txt = txt.replace('Maloja', 'Maloja / Maloggia')
  txt = txt.replace('Moësa', 'Moesa')
  txt = txt.replace('Wangen', 'Oberaargau')
  txt = txt.replace('Saanen', 'Obersimmental-Saanen')
  txt = txt.replace('Obersimmental', 'Obersimmental-Saanen')
  txt = txt.replace('Prättigau/Davos', 'Prättigau-Davos')
  txt = txt.replace("Pays-d'Enhaut", "Riviera-Pays-d'Enhaut")
  txt = txt.replace("Lac", "See / Lac")
  txt = txt.replace("Aarberg", "Seeland")
  txt = txt.replace("Büren", "Seeland")
  txt = txt.replace("Erlach", "Seeland")
  txt = txt.replace("Nidau", "Seeland")
  txt = txt.replace("Sankt Gallen", "St. Gallen")  
  txt = txt.replace("Obwalden", "Kanton Obwalden")     
  return txt

spatial_df['NAME_2'] = spatial_df['NAME_2'].apply(clean_text_spatial)

In [29]:
set(df.district_name) - set(spatial_df.NAME_2)

set()

In [30]:
spatial_df = spatial_df.rename(columns={'NAME_2':'district_name'})

In [31]:
spatial_df = spatial_df[['district_name', 'temp_mean', 'temp_stdev','pv_out_mea', 'pv_out_std']]

In [32]:
df = df.merge(spatial_df.drop_duplicates(subset=['district_name']), how='left', on='district_name')

In [33]:
#add ev points

In [34]:
ev_df = pd.read_csv(f'{data_path}ev_points.csv')

In [35]:
def clean_text_ev(txt):
  #txt = txt.str.replace('Bern', 'Bern-Mittelland')
  txt = txt.replace('Biel', 'Biel/Bienne')
  txt = txt.replace('Genève', 'Canton de Genève')
  txt = txt.replace('Frutigen \r\n', 'Frutigen-Niedersimmental')
  txt = txt.replace('Burgdorf', 'Emmental')
  txt = txt.replace('Signau', 'Emmental')
  txt = txt.replace('Trachselwald', 'Emmental')
  txt = txt.replace('Interlaken', 'Interlaken-Oberhasli')
  txt = txt.replace('Courtelary', 'Jura bernois')
  txt = txt.replace('Moutier', 'Jura bernois')
  txt = txt.replace('La Neuveville', 'Jura bernois')
  txt = txt.replace('Jura-North Vaudois', 'Jura-Nord vaudois')
  txt = txt.replace('Appenzell Innerrhoden', 'Kanton Appenzell Innerrhoden')
  txt = txt.replace('Bettingen', 'Kanton Basel-Stadt')
  txt = txt.replace('Basel', 'Kanton Basel-Stadt')
  txt = txt.replace('Riehen', 'Kanton Basel-Stadt')
  txt = txt.replace('Glarus', 'Kanton Glarus')
  txt = txt.replace('Nidwalden', 'Kanton Nidwalden')
  txt = txt.replace('Uri', 'Kanton Uri')
  txt = txt.replace('Zug', 'Kanton Zug')
  txt = txt.replace('Küssnacht', 'Küssnacht (SZ)')
  txt = txt.replace('Luzern', 'Luzern-Land')
  txt = txt.replace('Maloja', 'Maloja / Maloggia')
  txt = txt.replace('Moësa', 'Moesa')
  txt = txt.replace('Wangen', 'Oberaargau')
  txt = txt.replace('Saanen', 'Obersimmental-Saanen')
  txt = txt.replace('Obersimmental', 'Obersimmental-Saanen')
  txt = txt.replace('Prättigau/Davos', 'Prättigau-Davos')
  txt = txt.replace("Pays-d'Enhaut", "Riviera-Pays-d'Enhaut")
  txt = txt.replace("Lac", "See / Lac")
  txt = txt.replace("Aarberg", "Seeland")
  txt = txt.replace("Büren", "Seeland")
  txt = txt.replace("Erlach", "Seeland")
  txt = txt.replace("Nidau", "Seeland")
  txt = txt.replace("Sankt Gallen", "St. Gallen")  
  txt = txt.replace("Obwalden", "Kanton Obwalden")     
  return txt

ev_df['NAME_2'] = ev_df['NAME_2'].apply(clean_text_ev)

In [36]:
set(df.district_name) - set(ev_df.NAME_2)

set()

In [37]:
ev_df = ev_df.rename(columns={'NAME_2':'district_name'})

In [38]:
ev_df = ev_df[['district_name', 'NUMPOINTS']]

In [39]:
df = df.merge(ev_df.drop_duplicates(subset=['district_name']), how='left', on='district_name')

In [40]:
df.to_csv(data_path + 'clean_data.csv')